In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, BatchNormalization
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt


In [ ]:
cifar100 = tf.keras.datasets.cifar100
(x_train, y_train), (x_test, y_test) = cifar100.load_data()


x_train = x_train.astype('float32') / 255.0
x_test = x_test.astype('float32') / 255.0

input_shape = x_train.shape[1:]  # (32, 32, 3)


169001437/169001437 ━━━━━━━━━━━━━━━━━━━━ 3s 0us/step


In [ ]:
def build_improved_autoencoder(input_shape):
    # Encoder
    inputs = Input(shape=input_shape)
    x = Conv2D(64, (3, 3), activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D((2, 2), padding='same')(x)

    x = Conv2D(256, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    encoded = MaxPooling2D((2, 2), padding='same')(x)

    # Decoder
    x = Conv2D(256, (3, 3), activation='relu', padding='same')(encoded)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)

    x = Conv2D(128, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)

    x = Conv2D(64, (3, 3), activation='relu', padding='same')(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)

    decoded = Conv2D(3, (3, 3), activation='sigmoid', padding='same')(x)  # Output with 3 channels for RGB

    # Autoencoder and Encoder Models
    autoencoder = Model(inputs, decoded)
    encoder = Model(inputs, encoded)

    return autoencoder, encoder


In [ ]:
autoencoder, encoder = build_improved_autoencoder(input_shape)
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(x_train, x_train, epochs=30, batch_size=64, validation_data=(x_test, x_test))

Epoch 1/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 26s 21ms/step - loss: 0.0153 - val_loss: 0.0065
Epoch 2/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 31s 14ms/step - loss: 0.0058 - val_loss: 0.0059
Epoch 3/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - loss: 0.0048 - val_loss: 0.0046
Epoch 4/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 0.0043 - val_loss: 0.0044
Epoch 5/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - loss: 0.0038 - val_loss: 0.0038
Epoch 6/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - loss: 0.0035 - val_loss: 0.0031
Epoch 7/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 15ms/step - loss: 0.0032 - val_loss: 0.0030
Epoch 8/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - loss: 0.0031 - val_loss: 0.0029
Epoch 9/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 0.0029 - val_loss: 0.0030
Epoch 10/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 20s 14ms/step - loss: 0.0027 - val_loss: 0.0029
Epoch 11/30
782/782 ━━━━━━━━━━━━━━━━━━━━ 21s 14ms/step - loss: 0.0027 - val_loss: 0.0025
Epoch 12/30
782/782 ━━━━━━━━━━

In [ ]:
def evaluate_similarity(encoder, query_image, dataset, ground_truth_labels):

    query_embedding = encoder.predict(np.expand_dims(query_image, axis=0))
    dataset_embeddings = encoder.predict(dataset)


    query_embedding = query_embedding.reshape(-1)
    dataset_embeddings = dataset_embeddings.reshape(dataset_embeddings.shape[0], -1)


    similarities = cosine_similarity(query_embedding.reshape(1, -1), dataset_embeddings)
    similarities = similarities.flatten()


    sorted_indices = np.argsort(similarities)[::-1]


    query_label = ground_truth_labels[np.argmax(similarities)]


    relevant_labels = [1 if ground_truth_labels[i] == query_label else 0 for i in sorted_indices]


    top_k = 5
    retrieved_labels = relevant_labels[:top_k]


    precision = np.sum(retrieved_labels) / top_k
    recall = np.sum(retrieved_labels) / np.sum(relevant_labels)
    retrieval_accuracy = np.mean(retrieved_labels)

    return precision, recall, retrieval_accuracy


In [ ]:

query_image = x_test[0]
dataset = x_test
ground_truth_labels = y_test.flatten()


precision, recall, retrieval_accuracy = evaluate_similarity(encoder, query_image, dataset, ground_truth_labels)


print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Retrieval Accuracy: {retrieval_accuracy:.4f}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 206ms/step
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step
Precision: 0.2000
Recall: 0.0100
Retrieval Accuracy: 0.2000


In [ ]:
autoencoder.save('autoencoder_model_CIFAR_t2_cf.h5')
encoder.save('encoder_model_CIFAR_t2_cf.h5')

In [ ]:
loaded_autoencoder = tf.keras.models.load_model('autoencoder_model_CIFAR_t2.h5')
loaded_encoder = tf.keras.models.load_model('encoder_model_CIFAR_t2.h5')

precision, recall, retrieval_accuracy = evaluate_similarity(loaded_encoder, query_image, dataset, ground_truth_labels)

print(f"Precision: {precision:.4f}")
print(f"Recall: {recall:.4f}")
print(f"Retrieval Accuracy: {retrieval_accuracy:.4f}")


Precision: 0.2000
Recall: 0.0100
Retrieval Accuracy: 0.2000
